# Overview

This examples uses symbolic execution to solve the crackme binary.

Compile the binary with `gcc crackme.c -no-pie -o crackme`

In [20]:
program_directory = "/home/pac/Desktop/labs/lab7/crackme/"
program_path = program_directory + "crackme"

In [21]:
import os
print(os.popen("gdb " + program_path + " -batch -ex 'disassemble main'").read())

Dump of assembler code for function main:
   0x000000000040061f <+0>:	push   rbp
   0x0000000000400620 <+1>:	mov    rbp,rsp
   0x0000000000400623 <+4>:	sub    rsp,0x20
   0x0000000000400627 <+8>:	mov    DWORD PTR [rbp-0x14],edi
   0x000000000040062a <+11>:	mov    QWORD PTR [rbp-0x20],rsi
   0x000000000040062e <+15>:	mov    rax,QWORD PTR fs:0x28
   0x0000000000400637 <+24>:	mov    QWORD PTR [rbp-0x8],rax
   0x000000000040063b <+28>:	xor    eax,eax
   0x000000000040063d <+30>:	lea    rdi,[rip+0x10e]        # 0x400752
   0x0000000000400644 <+37>:	mov    eax,0x0
   0x0000000000400649 <+42>:	call   0x400500 <printf@plt>
   0x000000000040064e <+47>:	lea    rax,[rbp-0xc]
   0x0000000000400652 <+51>:	mov    rsi,rax
   0x0000000000400655 <+54>:	lea    rdi,[rip+0x107]        # 0x400763
   0x000000000040065c <+61>:	mov    eax,0x0
   0x0000000000400661 <+66>:	call   0x400510 <__isoc99_scanf@plt>
   0x0000000000400666 <+71>:	mov    eax,DWORD PTR [rbp-0xc]
   0x0000000000400669 <+74>:	cmp    eax,0x4

In [22]:
# import the python system and angr libraries

import angr
import sys

In [23]:
# load the challenge binary
proj = angr.Project(program_path)

main_address = proj.loader.find_symbol("main").rebased_addr

# recovered through objdump, gdb, radare2, or some dissassembler the offset
# to the start of the basic block that calls the print_flag function
target_address_offset = 81 # start of basic block with call to print_flag
target_address = main_address + target_address_offset

In [24]:
# start the program state at the beginning of main
state = proj.factory.entry_state(addr=main_address)
sm = proj.factory.simulation_manager(state)

In [28]:
# symbolically explore the program, solving for inputs that satisfy paths to the target address
sm.explore(find=target_address)
while len(sm.found) == 0:
    sm.step()

# if the solver found a way to reach the target address then print the input
if (len(sm.found) > 0):
    print("Found an input to satisfy the path constraints to reach target address!")
    found_input = sm.found[0].posix.dumps(sys.stdin.fileno())
    print(found_input)
    with open("analysis_result", "wb") as fp:
        fp.write(found_input)
else:
    print("Unable to compute an input that satisfies the path constraints to reach the target address")

Found an input to satisfy the path constraints to reach target address!
b'0000001234'


In [27]:
# test the computed input to make the program print the flag

command = program_path + " < " + program_directory + "analysis_result"
print(os.popen(command).read())

Enter Password: flag{program_analysis_is_fun}
